In [3]:
from skimage.io import imread
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from keras.layers import Input, Dense, Convolution2D, MaxPooling2D, UpSampling2D, Convolution2DTranspose, Add
from keras.models import Model, Sequential
from keras.preprocessing.image import ImageDataGenerator
import keras.optimizers as optimizers
#dimension of target image
IMG_HEIGHT = 576
IMG_WIDTH = 720
BATCH_SIZE = 4

#DATA GENERATION
def load_img(path):
    img = imread(path)
    img = img/ 255
    return img

def img_generator(img_list,batch_size):
    while True:
        X_batch_input = []
        y_batch_input = []
        degraded_path =  'C:/Users/Mathieu/CESI/Projet/dataset_clean_degraded/degraded/'
        clean_path =  'C:/Users/Mathieu/CESI/Projet/dataset_clean_degraded/clean/'
        batch_path = np.random.choice(a = img_list, size= batch_size)
        for img_name in batch_path:
            current_X = load_img(degraded_path+img_name)
            current_y = load_img(clean_path+img_name)
            X_batch_input += [ current_X ]
            y_batch_input += [ current_y ]
        batch_X = np.array(X_batch_input)
        batch_y = np.array(y_batch_input)
        yield (batch_X, batch_y)

#MODEL DEFINITION
model = Sequential()
init = Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3))
level1_1 = Convolution2D(16, (3, 3), activation='relu', padding='same')(init)
level2_1 = Convolution2D(16, (3, 3), activation='relu', padding='same')(level1_1)
level2_2 = Convolution2DTranspose(16, (3, 3), activation='relu', padding='same')(level2_1)
level2 = Add()([level2_1, level2_2])
level1_2 = Convolution2DTranspose(16, (3, 3), activation='relu', padding='same')(level2)
level1 = Add()([level1_1, level1_2])
decoded = Convolution2D(3, (5, 5), activation='linear', padding='same')(level1)
model = Model(init, decoded)
adam = optimizers.Adam(lr=1e-3)
model.compile(optimizer=adam, loss='mse', metrics=['acc'])
model.summary()

#TRAINING
img_list = os.listdir('C:/Users/Mathieu/CESI/Projet/dataset_clean_degraded/degraded/')
gen = img_generator(img_list, BATCH_SIZE)

STEPS_PER_EPOCH = np.ceil(5500/BATCH_SIZE)
history = model.fit_generator(gen, steps_per_epoch=STEPS_PER_EPOCH, epochs=2)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 576, 720, 3)  0                                            
__________________________________________________________________________________________________
conv2d_7 (Conv2D)               (None, 576, 720, 16) 448         input_3[0][0]                    
__________________________________________________________________________________________________
conv2d_8 (Conv2D)               (None, 576, 720, 16) 2320        conv2d_7[0][0]                   
__________________________________________________________________________________________________
conv2d_transpose_5 (Conv2DTrans (None, 576, 720, 16) 2320        conv2d_8[0][0]                   
__________________________________________________________________________________________________
add_5 (Add

KeyboardInterrupt: 

In [4]:
import pickle
pickle.dump( model , open( "model-save-pickle-ESRCNN.p", "wb"))
pickle.dump( history , open( "history-save-pickle-ESRCNN.p", "wb"))

NameError: name 'history' is not defined